In [4]:
import os
import json
import time
import pyautogui

def get_key():
    with open("secretkey.json", "r") as json_file:
        return json.load(json_file)["api_key"]

try:
    pyautogui.getWindowsWithTitle("my_presentation.pptx")[0].close()
    time.sleep(0.1)

except IndexError:
    print("Window '.pptx' not found.")

Window '.pptx' not found.


In [5]:
import os
import requests
import base64

# Configuration
GPT4V_KEY = get_key()

# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")
user_input = input("Hi, What powerpoint content would you like to generate:\n")
sys_prompt = """Instruct:
You are to use the provided infomation given by users to generate content for a powerpoint slide.
Create a PowerPoint presentation with the following structure:

1. **Title Slide**:
   - Title: [Your main title]
   - Subtitle: [Your subtitle]

2. **Table of Contents Slide**:
   - Title: "Table of Contents"
   - Sections: [List of section titles and brief descriptions]

3. **Section Slides**:
   For each section in the Table of Contents, provide:
   - Section number and title
   - A choice between a 2-5 point system or a long-form text. If choosing a point system, provide the points in the form of (header, content). If choosing long-form text, provide a single content string.
   
Here is an example format:

**Title Slide**:
Title: ""
Subtitle: ""

**Table of Contents Slide**:
Title: "Table of Contents"
Sections:
1. 
2.

**Section Slides**:
1. Section number: 1
   Title: ""
   Content type: 3-point system
   Points:
   - Header: ""
     Content: ""
   - Header: ""
     Content: ""
   - Header: ""
     Content: ""

2. Section number: 2
   Title: ""
   Content type: long-form text
   Content: ""

... and so on for other sections.


Guideline:
Not more then 8 sections as well as table of content
For points keep it within 2-5 points
Ensure there is the same number of sections as there is in the table of contents
Return in json format so that it can be used in the following code

add_long_text_slide(prs, title, content)
add_side_by_side_text_slide(prs, heading, left_text, right_text)
add_content_slide(prs, title, points)
add_section_slide(prs, heading, subheading)
add_title_slide(prs, title_text, subtitle_text)

content = json.loads(data)
title_slide = content["title_slide"]
add_title_slide(prs, title_slide["title"], title_slide["subtitle"])

# Add the table of contents slide
toc_slide = content["table_of_contents"]
add_table_of_contents_slide(prs, toc_slide["title"], toc_slide["sections"])

# Add the section slides
sections = content["sections"]
for section in sections:
    add_section_slide(
        prs, section["section_number"], section["title"], section["subheading"]
    )
    if section["content_type"] == "points":
        points = [(item["header"], item["text"]) for item in section["content"]]
        add_content_slide(prs, section["title"], points)
    elif section["content_type"] == "long_text":
        add_long_text_slide(prs, section["title"], section["content"])
    elif section["content_type"] == "side_by_side":
        add_side_by_side_text_slide(
            prs,
            section["title"],
            section["content"]["left_text"],
            section["content"]["right_text"],
        )


Example:
{
    "title_slide": {
        "title": "Exploring the Ocean Depths",
        "subtitle": "A Journey into the Mysterious Underwater World"
    },
    "table_of_contents": {
        "title": "Table of Contents",
        "sections": [
            {"heading": "Introduction", "subheading": "Overview of Ocean Exploration"},
            {"heading": "Marine Life", "subheading": "Diverse Creatures of the Deep"},
            {"heading": "Underwater Ecosystems", "subheading": "Complex Habitats Beneath the Waves"},
            {"heading": "Human Impact", "subheading": "Effects of Human Activities on Oceans"},
            {"heading": "Conservation Efforts", "subheading": "Protecting Our Marine Environments"}
        ]
    },
    "sections": [
        {
            "section_number": 1,
            "title": "Introduction",
            "subheading": "Overview of Ocean Exploration",
            "content_type": "points",
            "content": [
                {"header": "History", "text": "The history of ocean exploration dates back to ancient civilizations that sailed the seas."},
                {"header": "Technological Advances", "text": "Modern technology, such as submarines and remotely operated vehicles (ROVs), has expanded our ability to explore the deep ocean."},
                {"header": "Importance", "text": "Ocean exploration is crucial for understanding marine biodiversity and the health of our planet."}
            ]
        },
        {
            "section_number": 2,
            "title": "Marine Life",
            "subheading": "Diverse Creatures of the Deep",
            "content_type": "long_text",
            "content": "The ocean is home to an incredible array of marine life, from the smallest plankton to the largest whales. Each species plays a vital role in the ocean's ecosystem, contributing to its complexity and resilience. Many of these creatures have adapted to the unique conditions of their underwater environment, displaying remarkable behaviors and characteristics."
        },
        {
            "section_number": 3,
            "title": "Underwater Ecosystems",
            "subheading": "Complex Habitats Beneath the Waves",
            "content_type": "side_by_side",
            "content": {
                "left_text": "Coral reefs are among the most diverse and productive ecosystems on Earth. They provide habitat for thousands of marine species and protect coastlines from erosion.",
                "right_text": "Deep-sea hydrothermal vents are another fascinating ecosystem. These vents release mineral-rich water, supporting unique communities of organisms that thrive in extreme conditions."
            }
        },
        {
            "section_number": 4,
            "title": "Human Impact",
            "subheading": "Effects of Human Activities on Oceans",
            "content_type": "points",
            "content": [
                {"header": "Pollution", "text": "Plastic waste, oil spills, and chemical runoff are major pollutants affecting marine life."},
                {"header": "Overfishing", "text": "Overfishing has depleted many fish populations, disrupting oceanic food chains."},
                {"header": "Climate Change", "text": "Rising sea temperatures and acidification are threatening coral reefs and other marine habitats."}
            ]
        },
        {
            "section_number": 5,
            "title": "Conservation Efforts",
            "subheading": "Protecting Our Marine Environments",
            "content_type": "long_text",
            "content": "Conservation efforts are critical to preserving the health of our oceans. Marine protected areas (MPAs) are established to safeguard important habitats and species. Additionally, international agreements and regulations aim to reduce pollution and promote sustainable fishing practices. Public awareness and education are also essential in fostering a sense of stewardship for the marine environment."
        }
    ]
}
"""

headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": sys_prompt,
                }
            ],
        },
        {
            "role": "assistant",
            "content": [{"type": "text", "text": "Hi, What powerpoint content would you like to generate"}],
        },
        {"role": "user", 
         "content": [{"type": "text", "text": user_input}]},
    ],
    "temperature": 0.7,
    "top_p": 0.95,
}

GPT4V_ENDPOINT = "https://intern2024openai.openai.azure.com/openai/deployments/gpt4-Omni/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

data = response.json()
print(data)

{'choices': [{'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}, 'finish_reason': 'stop', 'index': 0, 'logprobs': None, 'message': {'content': '```json\n{\n    "title_slide": {\n        "title": "Lore of Genshin Impact",\n        "subtitle": "The First Descender and the Primordial One"\n    },\n    "table_of_contents": {\n        "title": "Table of Contents",\n        "sections": [\n            {"heading": "Introduction", "subheading": "Overview of Genshin Impact\'s Lore"},\n            {"heading": "The First Descender", "subheading": "Who is the First Descender?"},\n            {"heading": "The Primordial One", "subheading": "Origin and Role of the Primordial One"},\n            {"heading": "Connection Between the First Descender and the Primordial One", "subheading": "Intertwined Fates"},\n            {

In [6]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import PP_ALIGN
from pptx.dml.color import RGBColor
import re

def remove_all_placeholders(slide):
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)


def add_title_slide(prs, title_text, subtitle_text):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add title text box
    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(2), slide_width - Inches(2), Inches(1.5)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(54)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subtitle text box
    subtitle_shape = slide.shapes.add_textbox(
        Inches(1), Inches(4), slide_width - Inches(2), Inches(1)
    )
    subtitle_frame = subtitle_shape.text_frame
    subtitle_frame.text = subtitle_text
    subtitle_frame.paragraphs[0].font.size = Pt(32)
    subtitle_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subtitle_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105) 


def add_table_of_contents_slide(prs, title_text, sections):
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)
    remove_all_placeholders(slide)

    title_shape = slide.shapes.add_textbox(
        Inches(0.5), Inches(0.5), Inches(12.33), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title_text
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].font.color.rgb = RGBColor(192, 192, 192)

    left_positions = [Inches(0.5), Inches(6.5)]
    top_start = Inches(1.5)
    space_between = Inches(1.5)
    max_per_column = 4

    def adjust_text_size(text_frame, max_width):
        for paragraph in text_frame.paragraphs:
            run = paragraph.runs[0]
            while run.font.size is None or run.font.size.pt > 10:
                if run.font.size is not None and text_frame.auto_size:
                    break
                if run.font.size is None:
                    run.font.size = Pt(20)
                else:
                    run.font.size -= Pt(1)

    for i, section in enumerate(sections):
        heading = section["heading"]
        subheading = section["subheading"]

        column = i // max_per_column
        row = i % max_per_column

        left = left_positions[column]
        top = top_start + row * space_between

        number_shape = slide.shapes.add_textbox(left, top, Inches(1.0), Inches(1.0))
        number_frame = number_shape.text_frame
        number_frame.text = f"{i + 1:02}"
        number_frame.paragraphs[0].font.size = Pt(40)
        number_frame.paragraphs[0].font.bold = True
        number_frame.paragraphs[0].font.color.rgb = RGBColor(89, 89, 89)

        heading_shape = slide.shapes.add_textbox(
            left + Inches(1.0), top, Inches(5.0), Inches(0.5)
        )
        heading_frame = heading_shape.text_frame
        heading_frame.text = heading
        heading_frame.paragraphs[0].font.size = Pt(20)
        heading_frame.paragraphs[0].font.bold = True
        adjust_text_size(heading_frame, Inches(5.0))

        subheading_shape = slide.shapes.add_textbox(
            left + Inches(1.0), top + Inches(0.5), Inches(5.0), Inches(1.0)
        )
        subheading_frame = subheading_shape.text_frame
        subheading_frame.text = subheading
        subheading_frame.word_wrap = True
        subheading_frame.paragraphs[0].font.size = Pt(14)
        subheading_frame.paragraphs[0].font.color.rgb = RGBColor(128, 128, 128)
        adjust_text_size(subheading_frame, Inches(5.0))


def add_section_slide(prs, section_number, heading, subheading):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    remove_all_placeholders(slide)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add section number
    number_shape = slide.shapes.add_textbox(
        Inches(4.5), Inches(1.5), Inches(4.0), Inches(1.0)
    )
    number_frame = number_shape.text_frame
    number_frame.text = f"{section_number:02}"
    number_frame.paragraphs[0].font.size = Pt(100)
    number_frame.paragraphs[0].font.bold = True
    number_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add heading text box
    heading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(3.5), slide_width - Inches(2), Inches(1.5)
    )
    heading_frame = heading_shape.text_frame
    heading_frame.text = heading
    heading_frame.paragraphs[0].font.size = Pt(54)
    heading_frame.paragraphs[0].font.bold = True
    heading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Add subheading text box
    subheading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(5), slide_width - Inches(2), Inches(1)
    )
    subheading_frame = subheading_shape.text_frame
    subheading_frame.text = subheading
    subheading_frame.paragraphs[0].font.size = Pt(32)
    subheading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER
    subheading_frame.paragraphs[0].font.color.rgb = RGBColor(105, 105, 105)


def add_content_slide(prs, title, points):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add title text box
    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(0.5), slide_width - Inches(2), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    num_points = len(points)
    assert 2 <= num_points <= 5, "Number of points must be between 2 and 5"

    top_start = Inches(2)
    left_margin = Inches(1)
    col_width = (slide_width - 2 * left_margin) / 2

    if num_points == 2:
        top = top_start
        for i, (header, content) in enumerate(points):
            left = left_margin + i * col_width

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(0.5), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 3:
        top = top_start
        col_width = (slide_width - 2 * left_margin) / 3
        for i, (header, content) in enumerate(points):
            left = left_margin + i * col_width

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top+ Inches(0.75), col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(1.25), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 4:
        top_positions = [top_start, top_start + Inches(2.5)]
        for i, (header, content) in enumerate(points):
            left = left_margin + (i % 2) * col_width
            top = top_positions[i // 2]

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                left, top + Inches(0.5), col_width, Inches(1.5)
            )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    elif num_points == 5:
        top_positions = [top_start, top_start + Inches(3.5)]
        for i, (header, content) in enumerate(points):
            if i == 2:  # Middle item for 5 points
                left = (slide_width - col_width) / 2
                top = top_start + Inches(1.75)
            else:
                left = left_margin + (i % 2) * col_width
                if i < 2:
                    top = top_positions[0]
                else:
                    top = top_positions[1]

            # Add header text box
            header_shape = slide.shapes.add_textbox(left, top, col_width, Inches(0.5))
            header_frame = header_shape.text_frame
            header_frame.text = header
            header_frame.paragraphs[0].font.size = Pt(24)
            header_frame.paragraphs[0].font.bold = True
            header_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

            # Add content text box
            content_shape = slide.shapes.add_textbox(
                    left, top + Inches(0.5), col_width, Inches(1.5)
                )
            content_frame = content_shape.text_frame
            content_frame.text = content
            content_frame.word_wrap = True
            content_frame.paragraphs[0].font.size = Pt(18)
            content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


def add_side_by_side_text_slide(prs, heading, left_text, right_text):
    # Add a blank slide layout
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)

    # Remove any placeholders
    for shape in slide.shapes:
        if shape.is_placeholder:
            sp = shape._element
            sp.getparent().remove(sp)

    # Constants for positioning
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    # Add heading text box
    heading_shape = slide.shapes.add_textbox(
        Inches(1), Inches(0.5), slide_width - Inches(2), Inches(1)
    )
    heading_frame = heading_shape.text_frame
    heading_frame.text = heading
    heading_frame.paragraphs[0].font.size = Pt(32)
    heading_frame.paragraphs[0].font.bold = True
    heading_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    # Calculate positions for the two text boxes
    col_width = (slide_width - 3 * Inches(1)) / 2
    left_margin = Inches(1)
    top_margin = Inches(1.5)

    # Add left text box
    left_shape = slide.shapes.add_textbox(
        left_margin, top_margin + Inches(1), col_width, slide_height - Inches(2.5)
    )
    left_frame = left_shape.text_frame
    left_frame.text = left_text
    left_frame.word_wrap = True
    left_frame.paragraphs[0].font.size = Pt(18)
    left_frame.paragraphs[0].alignment = PP_ALIGN.LEFT

    # Add right text box
    right_shape = slide.shapes.add_textbox(
        left_margin + col_width + Inches(1),
        top_margin + Inches(1),
        col_width,
        slide_height - Inches(2.5),
    )
    right_frame = right_shape.text_frame
    right_frame.text = right_text
    right_frame.word_wrap = True
    right_frame.paragraphs[0].font.size = Pt(18)
    right_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


def add_long_text_slide(prs, title, content):
    slide_layout = prs.slide_layouts[6]  # Use a blank layout
    slide = prs.slides.add_slide(slide_layout)
    remove_all_placeholders(slide)

    slide_width = prs.slide_width
    slide_height = prs.slide_height

    title_shape = slide.shapes.add_textbox(
        Inches(1), Inches(0.5), slide_width - Inches(2), Inches(1)
    )
    title_frame = title_shape.text_frame
    title_frame.text = title
    title_frame.paragraphs[0].font.size = Pt(32)
    title_frame.paragraphs[0].font.bold = True
    title_frame.paragraphs[0].alignment = PP_ALIGN.CENTER

    content_shape = slide.shapes.add_textbox(
        Inches(1), Inches(1.5), slide_width - Inches(2), slide_height - Inches(2)
    )
    content_frame = content_shape.text_frame
    content_frame.text = content
    content_frame.word_wrap = True
    content_frame.paragraphs[0].font.size = Pt(18)
    content_frame.paragraphs[0].alignment = PP_ALIGN.LEFT


def extract_json_text(text):
    pattern = r"```json(.*?)```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None


def main():

    prs = Presentation()

    # Set slide width and height for 16:9 aspect ratio
    prs.slide_width = Inches(13.33)
    prs.slide_height = Inches(7.5)
    print(data["choices"][0]["message"]["content"])
    string_content = extract_json_text(data["choices"][0]["message"]["content"])
    print(string_content)

    content = json.loads(string_content)

    # Add the title slide
    title_slide = content["title_slide"]
    add_title_slide(prs, title_slide["title"], title_slide["subtitle"])

    # Add the table of contents slide
    toc_slide = content["table_of_contents"]
    print(toc_slide)
    add_table_of_contents_slide(prs, toc_slide["title"], toc_slide["sections"])

    # Add the section slides
    sections = content["sections"]
    for section in sections:
        add_section_slide(
            prs, section["section_number"], section["title"], section["subheading"]
        )
        if section["content_type"] == "points":
            points = [(item["header"], item["text"]) for item in section["content"]]
            add_content_slide(prs, section["title"], points)
        elif section["content_type"] == "long_text":
            add_long_text_slide(prs, section["title"], section["content"])
        elif section["content_type"] == "side_by_side":
            add_side_by_side_text_slide(
                prs,
                section["title"],
                section["content"]["left_text"],
                section["content"]["right_text"],
            )

    # Save the presentation
    prs.save("presentation.pptx")

    print("Presentation created successfully")


if __name__ == "__main__":
    main()

```json
{
    "title_slide": {
        "title": "Lore of Genshin Impact",
        "subtitle": "The First Descender and the Primordial One"
    },
    "table_of_contents": {
        "title": "Table of Contents",
        "sections": [
            {"heading": "Introduction", "subheading": "Overview of Genshin Impact's Lore"},
            {"heading": "The First Descender", "subheading": "Who is the First Descender?"},
            {"heading": "The Primordial One", "subheading": "Origin and Role of the Primordial One"},
            {"heading": "Connection Between the First Descender and the Primordial One", "subheading": "Intertwined Fates"},
            {"heading": "Impact on Teyvat", "subheading": "How These Entities Shaped the World"},
            {"heading": "Speculations and Theories", "subheading": "Fan Theories and Speculations"}
        ]
    },
    "sections": [
        {
            "section_number": 1,
            "title": "Introduction",
            "subheading": "Overview of Gen

In [7]:
os.startfile("presentation.pptx")